In [1]:
import pandas as pd

In [ ]:
# # code used to create the data splits
# CFS = pd.read_csv('../data/CoFE/CFS_without_debates_CFET.tsv', sep='\t')
# proposals = pd.read_csv('../data/CoFE/prop_CF_all.tsv', sep='\t')
# CoFE_train = pd.merge(proposals, CFS.drop('Topic', axis=1), left_on='id', right_on='id_prop', suffixes=('_prop', '_cm'))
# CoFE_train = CoFE_train.loc[:, ~CoFE_train.columns.duplicated()]
# CoFE_train.to_csv('../data/CoFE/CoFE_train.csv', index=None)